<a href="https://colab.research.google.com/github/rodrigozago/compiladores/blob/master/Lexico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tabulate import tabulate
import pandas as pd
import numpy as np
import re
from IPython.display import display

Analisador
==================

Definição da tabela de simbolos
--------------------

Classe que define a tabela de simbolos.

symTable: Estrutura do tipo dict para armazenar os tokens:

```{ lexema<string> : token<Token> }```

In [2]:
class SymTable:
    def __init__(self):
        self.symTable = dict();
    def addNewEntry(self, token):
        if(token.lexema not in self.symTable):
            self.symTable[token.lexema] = token
        return token
    def getEntry(self, lexema):
        if(lexema in self.symTable):
            return self.symTable[lexema]
        else:
            return None

Definição da classe Token
--------------------------

In [3]:
class Token:
    def __init__(self, lexema, token, tipo):
        self.lexema = lexema
        self.token = token
        self.tipo = tipo
    
    def __str__(self):
        return "Lexema: {} Token: {} Tipo: {}".format(self.lexema, self.token, self.tipo)

    def getList(self):
        return [self.lexema, self.token, self.tipo]
    
    def get(self):
        return self.token

Adicionando palavras chave da linguagem na tabela de simbolos
---------------------------------------

In [4]:
# Creating symbol table
table = SymTable()

# Add keywords into symbol table
try:
    table.addNewEntry(Token('inicio', 'inicio', '-'))
    table.addNewEntry(Token('varinicio', 'varinicio', '-'))
    table.addNewEntry(Token('varfim', 'varfim', '-'))
    table.addNewEntry(Token('escreva', 'escreva', '-'))
    table.addNewEntry(Token('leia', 'leia', '-'))
    table.addNewEntry(Token('se', 'se', '-'))
    table.addNewEntry(Token('entao', 'entao', '-'))
    table.addNewEntry(Token('fimse', 'fimse', '-'))
    table.addNewEntry(Token('fim', 'fim', '-'))
    table.addNewEntry(Token('inteiro', 'inteiro', '-'))
    table.addNewEntry(Token('lit', 'lit', '-'))
    table.addNewEntry(Token('real', 'real', '-'))    
except Exception as e:
    print("erro: {}".format(e))



In [5]:
for key in table.symTable:
    print(table.symTable[key])

Lexema: inicio Token: inicio Tipo: -
Lexema: varinicio Token: varinicio Tipo: -
Lexema: varfim Token: varfim Tipo: -
Lexema: escreva Token: escreva Tipo: -
Lexema: leia Token: leia Tipo: -
Lexema: se Token: se Tipo: -
Lexema: entao Token: entao Tipo: -
Lexema: fimse Token: fimse Tipo: -
Lexema: fim Token: fim Tipo: -
Lexema: inteiro Token: inteiro Tipo: -
Lexema: lit Token: lit Tipo: -
Lexema: real Token: real Tipo: -


Tabela dos estados finais
-------------

In [6]:
state_tokens = {
    1: 'Num',
    2: 'Num',
    3: 'Num',
    6: 'Num',
    8: 'Literal',
    9: 'id',
    11: 'Comentário',
    12: 'EOF',
    13: 'OPR',
    14: 'OPR',
    15: 'OPR',
    16: 'OPR',
    17: 'RCB',
    18: 'OPR',
    19: 'OPM',
    20: 'AB_P',
    21: 'FC_P',
    22: 'PT_V'
}

Definição do autômato finito
-----------------------------

Definicao da class DFA

In [7]:
class DFA:
    current_state = None;
    def __init__(self, states, alphabet, transition_function, start_state, accept_states):
        self.states = states;
        self.alphabet = alphabet;
        self.transition_function = transition_function;
        self.start_state = start_state;
        self.accept_states = accept_states;
        self.current_state = start_state;
        return;
    
    def transition_to_state_with_input(self, input_value):
        if((self.current_state, input_value) not in self.transition_function.keys()):
            if((self.current_state, '@') in self.transition_function.keys()):
                self.current_state = self.transition_function[(self.current_state, '@')];
                return True;  
            return False;
        previous_state = self.current_state;
        self.current_state = self.transition_function[(self.current_state, input_value)];
        # debug do automato
        # print("({}, {}) -> {}".format(previous_state, input_value, self.current_state))
        return True;
            
            
    def addToSymbolTable(self, lexema, token):
        result = table.getEntry(lexema)
        if result:
            return result
        else: 
            return table.addNewEntry(token)
        
    
    def in_accept_state(self):
        return self.current_state in accept_states;
    
    def go_to_initial_state(self):
        self.current_state = self.start_state;
        return;

Definindo função de transição

In [8]:
tf = dict();

# Begin tf[(0, D)] = 1;
tf[(0, '0')] = 1;
tf[(0, '1')] = 1;
tf[(0, '2')] = 1;
tf[(0, '3')] = 1;
tf[(0, '4')] = 1;
tf[(0, '5')] = 1;
tf[(0, '6')] = 1;
tf[(0, '7')] = 1;
tf[(0, '8')] = 1;
tf[(0, '9')] = 1;
# End tf[(0, D)] = 1;

# Begin tf[(1, D)] = 1;
tf[(1, '0')] = 1;
tf[(1, '1')] = 1;
tf[(1, '2')] = 1;
tf[(1, '3')] = 1;
tf[(1, '4')] = 1;
tf[(1, '5')] = 1;
tf[(1, '6')] = 1;
tf[(1, '7')] = 1;
tf[(1, '8')] = 1;
tf[(1, '9')] = 1;
# End tf[(1, D)] = 1;

#TODO RESOLVER \. 
tf[(1, '.')] = 2;
tf[(1, 'E')] = 4;
tf[(1, 'e')] = 4;

# Begin tf[(2, D)] = 3;
tf[(2, '0')] = 3;
tf[(2, '1')] = 3;
tf[(2, '2')] = 3;
tf[(2, '3')] = 3;
tf[(2, '4')] = 3;
tf[(2, '5')] = 3;
tf[(2, '6')] = 3;
tf[(2, '7')] = 3;
tf[(2, '8')] = 3;
tf[(2, '9')] = 3;
# End tf[(2, D)] = 3;

# TODO VERIFICAR COM O PRATES
# tf[(2, "QUALQUER COISA")] = 23;

# Begin tf[(3, D)] = 3;
tf[(3, '0')] = 3;
tf[(3, '1')] = 3;
tf[(3, '2')] = 3;
tf[(3, '3')] = 3;
tf[(3, '4')] = 3;
tf[(3, '5')] = 3;
tf[(3, '6')] = 3;
tf[(3, '7')] = 3;
tf[(3, '8')] = 3;
tf[(3, '9')] = 3;
# End tf[(3, D)] = 3;

tf[(3, 'E')] = 4;
tf[(3, 'e')] = 4;
tf[(4, '+')] = 5;
tf[(4, '-')] = 5;

# Begin tf[(4, D)] = 6;
tf[(4, '0')] = 6;
tf[(4, '1')] = 6;
tf[(4, '2')] = 6;
tf[(4, '3')] = 6;
tf[(4, '4')] = 6;
tf[(4, '5')] = 6;
tf[(4, '6')] = 6;
tf[(4, '7')] = 6;
tf[(4, '8')] = 6;
tf[(4, '9')] = 6;
# End tf[(4, D)] = 6;

# TODO VERIFICAR COM O PRATES (ACONTECEU ERRO?)
# tf[(4, 'QUALQUER COISA')] = 23;

# Begin tf[(5, D)] = 6;
tf[(5, '0')] = 6;
tf[(5, '1')] = 6;
tf[(5, '2')] = 6;
tf[(5, '3')] = 6;
tf[(5, '4')] = 6;
tf[(5, '5')] = 6;
tf[(5, '6')] = 6;
tf[(5, '7')] = 6;
tf[(5, '8')] = 6;
tf[(5, '9')] = 6;
# End tf[(5, D)] = 6;

# TODO VERIFICAR COM O PRATES (ACONTECEU ERRO?)
# tf[(5, 'QUALQUER COISA')] = 23;

# Begin tf[(6, D)] = 6;
tf[(6, '0')] = 6;
tf[(6, '1')] = 6;
tf[(6, '2')] = 6;
tf[(6, '3')] = 6;
tf[(6, '4')] = 6;
tf[(6, '5')] = 6;
tf[(6, '6')] = 6;
tf[(6, '7')] = 6;
tf[(6, '8')] = 6;
tf[(6, '9')] = 6;
# End tf[(6, D)] = 6;

tf[(0, '"')] = 7;
tf[(7, '@')] = 7; # aqui o @ substitui ., aceita tudo 
tf[(7, '"')] = 8;

tf[(0, ' ')] = 23;
tf[(0, '\n')] = 23;
tf[(0, '\t')] = 23;

# Begin tf[(0, L)] = 9;
tf[(0, 'a')] = 9;
tf[(0, 'b')] = 9;
tf[(0, 'c')] = 9;
tf[(0, 'd')] = 9;
tf[(0, 'e')] = 9;
tf[(0, 'f')] = 9;
tf[(0, 'g')] = 9;
tf[(0, 'h')] = 9;
tf[(0, 'i')] = 9;
tf[(0, 'j')] = 9;
tf[(0, 'k')] = 9;
tf[(0, 'l')] = 9;
tf[(0, 'm')] = 9;
tf[(0, 'n')] = 9;
tf[(0, 'o')] = 9;
tf[(0, 'p')] = 9;
tf[(0, 'q')] = 9;
tf[(0, 'r')] = 9;
tf[(0, 's')] = 9;
tf[(0, 't')] = 9;
tf[(0, 'u')] = 9;
tf[(0, 'v')] = 9;
tf[(0, 'w')] = 9;
tf[(0, 'x')] = 9;
tf[(0, 'y')] = 9;
tf[(0, 'z')] = 9;
tf[(0, 'A')] = 9;
tf[(0, 'B')] = 9;
tf[(0, 'C')] = 9;
tf[(0, 'D')] = 9;
tf[(0, 'E')] = 9;
tf[(0, 'F')] = 9;
tf[(0, 'G')] = 9;
tf[(0, 'H')] = 9;
tf[(0, 'I')] = 9;
tf[(0, 'J')] = 9;
tf[(0, 'K')] = 9;
tf[(0, 'L')] = 9;
tf[(0, 'M')] = 9;
tf[(0, 'N')] = 9;
tf[(0, 'O')] = 9;
tf[(0, 'P')] = 9;
tf[(0, 'Q')] = 9;
tf[(0, 'R')] = 9;
tf[(0, 'S')] = 9;
tf[(0, 'T')] = 9;
tf[(0, 'U')] = 9;
tf[(0, 'V')] = 9;
tf[(0, 'W')] = 9;
tf[(0, 'X')] = 9;
tf[(0, 'Y')] = 9;
tf[(0, 'Z')] = 9;
# End tf[(0, L)] = 9;

# Begin tf[(9, L)] = 9;
tf[(9, 'a')] = 9;
tf[(9, 'b')] = 9;
tf[(9, 'c')] = 9;
tf[(9, 'd')] = 9;
tf[(9, 'e')] = 9;
tf[(9, 'f')] = 9;
tf[(9, 'g')] = 9;
tf[(9, 'h')] = 9;
tf[(9, 'i')] = 9;
tf[(9, 'j')] = 9;
tf[(9, 'k')] = 9;
tf[(9, 'l')] = 9;
tf[(9, 'm')] = 9;
tf[(9, 'n')] = 9;
tf[(9, 'o')] = 9;
tf[(9, 'p')] = 9;
tf[(9, 'q')] = 9;
tf[(9, 'r')] = 9;
tf[(9, 's')] = 9;
tf[(9, 't')] = 9;
tf[(9, 'u')] = 9;
tf[(9, 'v')] = 9;
tf[(9, 'w')] = 9;
tf[(9, 'x')] = 9;
tf[(9, 'y')] = 9;
tf[(9, 'z')] = 9;
tf[(9, 'A')] = 9;
tf[(9, 'B')] = 9;
tf[(9, 'C')] = 9;
tf[(9, 'D')] = 9;
tf[(9, 'E')] = 9;
tf[(9, 'F')] = 9;
tf[(9, 'G')] = 9;
tf[(9, 'H')] = 9;
tf[(9, 'I')] = 9;
tf[(9, 'J')] = 9;
tf[(9, 'K')] = 9;
tf[(9, 'L')] = 9;
tf[(9, 'M')] = 9;
tf[(9, 'N')] = 9;
tf[(9, 'O')] = 9;
tf[(9, 'P')] = 9;
tf[(9, 'Q')] = 9;
tf[(9, 'R')] = 9;
tf[(9, 'S')] = 9;
tf[(9, 'T')] = 9;
tf[(9, 'U')] = 9;
tf[(9, 'V')] = 9;
tf[(9, 'W')] = 9;
tf[(9, 'X')] = 9;
tf[(9, 'Y')] = 9;
tf[(9, 'Z')] = 9;
# End tf[(9, L)] = 9;



# Begin tf[(9, D)] = 9;
tf[(9, '0')] = 9;
tf[(9, '1')] = 9;
tf[(9, '2')] = 9;
tf[(9, '3')] = 9;
tf[(9, '4')] = 9;
tf[(9, '5')] = 9;
tf[(9, '6')] = 9;
tf[(9, '7')] = 9;
tf[(9, '8')] = 9;
tf[(9, '9')] = 9;
# End tf[(9, D)] = 9;

tf[(9, '_')] = 9;


tf[(0, '{')] = 10;
tf[(10, '@')] = 10; # aqui o @ substitui ., aceita tudo 
tf[(10, '}')] = 11;


tf[(0, '$')] = 12;

tf[(0, '>')] = 13;
tf[(13, '=')] = 14;

tf[(0, '<')] = 15;
tf[(15, '=')] = 14;
tf[(15, '>')] = 16;
tf[(15, '-')] = 17;

tf[(0, '=')] = 18;

# Begin tf[(0, OP)] = 19;
tf[(0, '+')] = 19;
tf[(0, '-')] = 19;
tf[(0, '*')] = 19;
tf[(0, '/')] = 19;
# End tf[(0, OP)] = 19;

tf[(0, '(')] = 20;
tf[(0, ')')] = 21;
tf[(0, ';')] = 22;

Definicao do alfabeto e estados

In [9]:
states = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23};
alphabet = {
    'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','y','x','z', 
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'X', 'Z',
    '0','1','2','3','4','5','6','7','8','9', 
    '+','-','*','/', 
    ' ','{','}','"',';','(',')','.', '>','<','='};
start_state = 0;
accept_states = {1, 3, 6, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23};

Definição do Analisador Léxico
----------------------------------------

In [10]:
class Scanner:
    dfa = None
    file = None
    rowList = None
    tokens = []
    errors = []
    i = 0
    j = 0
    end_of_analisys = False
    
    def __init__(self, dfa):  
        self.dfa = dfa
        self.file = open('fonte.alg')
        self.rowList = list(self.file)
        
    def getCursorPos(self):
        return [self.i, self.j];
        
    def restartAnalisys(self):
        self.end_of_analisys = False
        self.tokens = []
        self.errors = []
        self.i = 0 
        self.j = 0
        
    # Continue analisys until next token
    def getToken(self):
        if self.end_of_analisys:
            return Token("$", state_tokens[12], '-')
        self.dfa.go_to_initial_state();
        lexema = ""
        first = True
        while self.i < len(self.rowList):
            charList = list(self.rowList[self.i])
            # If is first iteration, get J from last stoped analisys
            if first:
                first = False
            else:
                # Else set j = 0 to first char on column
                self.j = 0
            while self.j < len(charList):
                
                didTransition = self.dfa.transition_to_state_with_input(charList[self.j]);
                # If is \n or \t or space or comment, ignore
                if(self.dfa.current_state is 23 or self.dfa.current_state is 11):
                    self.j = self.j + 1
                    self.dfa.go_to_initial_state()
                    lexema = ""
                    continue
                    
                if didTransition:
                    lexema = lexema + charList[self.j]
                    # Se for o ultimo lexema, caso seja erro, retornar eof, se não retornar o token e finalizar analise
                    if self.i is len(self.rowList)-1 and self.j is len(charList)-1:
                        # If is the last token
                        if self.dfa.in_accept_state():
                            #If has stopped in an accept state
                            token = Token(lexema, state_tokens[self.dfa.current_state], '-')
                            # Add to symbolTable if is ID 
                            if self.dfa.current_state is 9:
                                token = self.dfa.addToSymbolTable(lexema, token)
                            self.tokens.append(token)
                            self.dfa.go_to_initial_state()
                            lexema = ""
                            self.end_of_analisys = True
                            return token;
                        else:
                            self.errors.append([lexema, self.i+1, self.j+1])
                            print("!!! ERRO: Caracter {} não permitido. Na linha {} e coluna {}.".format(charList[self.j], self.i+1, self.j+1))
                            lexema = ""
                            self.dfa.go_to_initial_state()
                            return Token("$", state_tokens[12], '-')
                            
                else:
                    if self.dfa.in_accept_state():
                        token = Token(lexema, state_tokens[self.dfa.current_state], '-')
                        # Add to symbolTable if is ID 
                        if self.dfa.current_state is 9:
                            token = self.dfa.addToSymbolTable(lexema, token)
                        self.tokens.append(token)
                        self.dfa.go_to_initial_state()
                        lexema = ""
                        return token;
                    else: 
                        self.errors.append([lexema + charList[self.j], self.i+1, self.j+1])
                        print("!!! ERRO: Caracter {} não permitido. Na linha {} e coluna {}.".format(lexema + charList[self.j], self.i+1, self.j+1))
                        lexema = ""
                        self.dfa.go_to_initial_state()

                self.j = self.j + 1
            self.i = self.i + 1
            
            
    def doAnalisys(self):
        i = 0
        j = 0
        self.dfa.go_to_initial_state();
        tokens = []
        lexema = ""
        
        while i < len(self.rowList):
            charList = list(self.rowList[i])
            j = 0
            while j < len(charList):
                
                didTransition = self.dfa.transition_to_state_with_input(charList[j]);
    
                if(self.dfa.current_state is 23 or self.dfa.current_state is 11):
                    j = j + 1
                    self.dfa.go_to_initial_state()
                    lexema = ""
                    continue
                    
                if didTransition:
                    lexema = lexema + charList[j]
                    if i is len(self.rowList)-1 and j is len(charList)-1:
                        # If is the last token
                        if self.dfa.in_accept_state():
                            #If has stopped in an accept state
                            token = Token(lexema, state_tokens[self.dfa.current_state], '-')
                            print(token)
                            self.tokens.append(token)
                            self.dfa.go_to_initial_state()
                            lexema = ""
                        else:
                            self.errors.append([lexema, i+1, j+1])
                            print("!!! ERRO: Caracter {} não permitido. Na linha {} e coluna {}.".format(charList[j], i+1, j+1))
                            lexema = ""
                            self.dfa.go_to_initial_state()
                            
                else:
                    if self.dfa.in_accept_state():
                        token = Token(lexema, state_tokens[self.dfa.current_state], '-')
                        token = self.dfa.addToSymbolTable(lexema, token)
                        print(token)
                        self.tokens.append(token)
                        self.dfa.go_to_initial_state()
                        lexema = ""
                        continue
                    else: 
                        self.errors.append([lexema + charList[j], i+1, j+1])
                        print("!!! ERRO: Caracter {} não permitido. Na linha {} e coluna {}.".format(lexema + charList[j], i+1, j+1))
                        lexema = ""
                        self.dfa.go_to_initial_state()

                j = j + 1
            i = i + 1

In [11]:
d = DFA(states, alphabet, tf, start_state, accept_states);

In [12]:
scanner = Scanner(d)

In [13]:
scanner.restartAnalisys()

In [14]:
token = scanner.getToken()
print(token)
while token.token is not "EOF":
    token = scanner.getToken()
    print (token)

Lexema: inicio Token: inicio Tipo: -
Lexema: varinicio Token: varinicio Tipo: -
Lexema: A Token: id Tipo: -
Lexema: lit Token: lit Tipo: -
Lexema: ; Token: PT_V Tipo: -
Lexema: B Token: id Tipo: -
Lexema: inteiro Token: inteiro Tipo: -
Lexema: ; Token: PT_V Tipo: -
Lexema: D Token: id Tipo: -
Lexema: inteiro Token: inteiro Tipo: -
Lexema: ; Token: PT_V Tipo: -
Lexema: C Token: id Tipo: -
Lexema: real Token: real Tipo: -
Lexema: ; Token: PT_V Tipo: -
Lexema: varfim Token: varfim Tipo: -
Lexema: ; Token: PT_V Tipo: -
Lexema: escreva Token: escreva Tipo: -
Lexema: "Digite B" Token: Literal Tipo: -
Lexema: ; Token: PT_V Tipo: -
Lexema: leia Token: leia Tipo: -
Lexema: B Token: id Tipo: -
Lexema: ; Token: PT_V Tipo: -
Lexema: escreva Token: escreva Tipo: -
Lexema: "Digite A:" Token: Literal Tipo: -
Lexema: ; Token: PT_V Tipo: -
Lexema: leia Token: leia Tipo: -
Lexema: A Token: id Tipo: -
Lexema: ; Token: PT_V Tipo: -
Lexema: se Token: se Tipo: -
Lexema: ( Token: AB_P Tipo: -
Lexema: B Token

In [15]:
token = scanner.getToken()

In [16]:
token.get()

'EOF'

In [17]:
tokens_as_list = []
for token in scanner.tokens:
    tokens_as_list.append(token.getList())
print("\nTOKENS\n")
print("Foram reconhecidos {} tokens: \n".format(len(scanner.tokens)))
print(tabulate(tokens_as_list, headers=["LEXEMA","TOKEN", "TIPO"]))
if len(scanner.errors):
    print("\nCompilação finalizada com erros:\n")
    print("\nERROS\n")
    print(tabulate(scanner.errors, headers=["LEXEMA", "LINHA", "COLUNA"]))
else:
    print("\nCompilação finalizada.\n")


TOKENS

Foram reconhecidos 93 tokens: 

LEXEMA                TOKEN      TIPO
--------------------  ---------  ------
inicio                inicio     -
varinicio             varinicio  -
A                     id         -
lit                   lit        -
;                     PT_V       -
B                     id         -
inteiro               inteiro    -
;                     PT_V       -
D                     id         -
inteiro               inteiro    -
;                     PT_V       -
C                     id         -
real                  real       -
;                     PT_V       -
varfim                varfim     -
;                     PT_V       -
escreva               escreva    -
"Digite B"            Literal    -
;                     PT_V       -
leia                  leia       -
B                     id         -
;                     PT_V       -
escreva               escreva    -
"Digite A:"           Literal    -
;                     PT_V       -
leia  

Analisador Sintático:
================

Gramática

In [18]:
ruleRegex = re.compile('(\D+) (->) (.*)')

grammar = dict();

rules = ["P' -> P",
"P -> inicio V A",
"V -> varinicio LV",
"LV -> D LV",
"LV -> varfim ;",
"D -> id TIPO ;",
"TIPO -> int",
"TIPO -> real",
"TIPO -> lit",
"A -> ES A",
"ES -> leia id ;",
"ES -> escreva ARG ;",
"ARG -> literal",
"ARG -> num",
"ARG -> id",
"A -> CMD A",
"CMD -> id rcb LD ;",
"LD -> OPRD opm OPRD",
"LD -> OPRD",
"OPRD -> id",
"OPRD -> num",
"A -> COND A",
"COND -> CABEÇALHO CORPO",
"CABEÇALHO -> se ( EXP_R ) então",
"EXP_R -> OPRD opr OPRD",
"CORPO -> ES CORPO",
"CORPO -> CMD CORPO",
"CORPO -> COND CORPO",
"CORPO -> fimse",
"A -> fim"
]


numTokenGrammar = [
1,
3,
2,
2,
2,
3,
1,
1,
1,
2,
3,
3,
1,
1,
1,
2,
4,
3,
1,
1,
1,
2,
2,
5,
3,
2,
2,
2,
1,
1]

i = 0
for rule in rules:
    match = ruleRegex.match(rule)
    result = match.group(1)
    grammar[i] = dict(left=result, length=numTokenGrammar[i])
    print("{} left: {} length: {}".format(i, result, grammar[i]))
    i += 1
    


follow = {}
follow["P'"] = ['EOF'] # $ token fim de arquivo
follow["P"] = ['EOF'] # $
follow["V"] = ['fim', 'leia','escreva','id','se']
follow["LV"] = ['fim', 'leia','escreva','id','se']
follow["D"] = ['varfim','id']
follow["TIPO"] = ['PT_V']
follow["A"] = ['EOF'] # $
follow["ES"] = ['fim','leia','escreva','id','se','fimse']
follow["ARG"] = ['PT_V']
follow["CMD"] = ['fim','leia','escreva','id','se','fimse']
follow["LD"] = ['PT_V']
follow['OPRD'] = ['opm','PT_V','opr','FC_P']
follow['COND'] = ['fim','leia','escreva','id','se','fimse']
follow['CABEÇALHO'] = ['leia','escreva','id','fimse','se']
follow['EXP_R'] = ['FC_P']
follow['CORPO'] = ['fim','leia','escreva','id','se','fimse']

errors = {}
for i in range(59):
    errors[i] = None
errors[0] = "Era(m) esperado(s) o(s) token(s) {} ".format("inicio")
errors[2] = "Era(m) esperado(s) o(s) token(s) {} ".format("varinicio")
errors[3] = "Era(m) esperado(s) o(s) token(s) {} ".format("[fim, leia, escreva,id, se]")
errors[4] = "Era(m) esperado(s) o(s) token(s) {} ".format("[varfim, id]")
errors[10] = "Era(m) esperado(s) o(s) token(s) {} ".format("[id]")
errors[11] = "Era(m) esperado(s) o(s) token(s) {} ".format("[literal, num, id]")
errors[12] = "Era(m) esperado(s) o(s) token(s) {} ".format("<-")
errors[13] = "Era(m) esperado(s) o(s) token(s) {} ".format("fimse")
errors[14] = "Era(m) esperado(s) o(s) token(s) {} ".format("(")
errors[17] = "Era(m) esperado(s) o(s) token(s) {} ".format(";")
errors[18] = "Era(m) esperado(s) o(s) token(s) {} ".format("int,real, lit")
errors[22] = "Era(m) esperado(s) o(s) token(s) {} ".format(";")
errors[23] = "Era(m) esperado(s) o(s) token(s) {} ".format(";")
errors[36] = "Era(m) esperado(s) o(s) token(s) {} ".format(";")
errors[42] = "Era(m) esperado(s) o(s) token(s) {} ".format(";")
errors[43] = "Era(m) esperado(s) o(s) token(s) {} ".format("+ , -, *, / ")
errors[49] = "Era(m) esperado(s) o(s) token(s) {} ".format(")")
errors[50] = "Era(m) esperado(s) o(s) token(s) {} ".format("<, >, >= , <= , =, <>")
errors[53] = "Era(m) esperado(s) o(s) token(s) {} ".format("id, num")
errors[54] = "Era(m) esperado(s) o(s) token(s) {} ".format("entao")
errors
    

0 left: P' length: {'left': "P'", 'length': 1}
1 left: P length: {'left': 'P', 'length': 3}
2 left: V length: {'left': 'V', 'length': 2}
3 left: LV length: {'left': 'LV', 'length': 2}
4 left: LV length: {'left': 'LV', 'length': 2}
5 left: D length: {'left': 'D', 'length': 3}
6 left: TIPO length: {'left': 'TIPO', 'length': 1}
7 left: TIPO length: {'left': 'TIPO', 'length': 1}
8 left: TIPO length: {'left': 'TIPO', 'length': 1}
9 left: A length: {'left': 'A', 'length': 2}
10 left: ES length: {'left': 'ES', 'length': 3}
11 left: ES length: {'left': 'ES', 'length': 3}
12 left: ARG length: {'left': 'ARG', 'length': 1}
13 left: ARG length: {'left': 'ARG', 'length': 1}
14 left: ARG length: {'left': 'ARG', 'length': 1}
15 left: A length: {'left': 'A', 'length': 2}
16 left: CMD length: {'left': 'CMD', 'length': 4}
17 left: LD length: {'left': 'LD', 'length': 3}
18 left: LD length: {'left': 'LD', 'length': 1}
19 left: OPRD length: {'left': 'OPRD', 'length': 1}
20 left: OPRD length: {'left': 'OPRD

{0: 'Era(m) esperado(s) o(s) token(s) inicio ',
 1: None,
 2: 'Era(m) esperado(s) o(s) token(s) varinicio ',
 3: 'Era(m) esperado(s) o(s) token(s) [fim, leia, escreva,id, se] ',
 4: 'Era(m) esperado(s) o(s) token(s) [varfim, id] ',
 5: None,
 6: None,
 7: None,
 8: None,
 9: None,
 10: 'Era(m) esperado(s) o(s) token(s) [id] ',
 11: 'Era(m) esperado(s) o(s) token(s) [literal, num, id] ',
 12: 'Era(m) esperado(s) o(s) token(s) <- ',
 13: 'Era(m) esperado(s) o(s) token(s) fimse ',
 14: 'Era(m) esperado(s) o(s) token(s) ( ',
 15: None,
 16: None,
 17: 'Era(m) esperado(s) o(s) token(s) ; ',
 18: 'Era(m) esperado(s) o(s) token(s) int,real, lit ',
 19: None,
 20: None,
 21: None,
 22: 'Era(m) esperado(s) o(s) token(s) ; ',
 23: 'Era(m) esperado(s) o(s) token(s) ; ',
 24: None,
 25: None,
 26: None,
 27: None,
 28: None,
 29: None,
 30: None,
 31: None,
 32: None,
 33: None,
 34: None,
 35: None,
 36: 'Era(m) esperado(s) o(s) token(s) ; ',
 37: None,
 38: None,
 39: None,
 40: None,
 41: None,

Definição do analisador sintático

In [19]:
class Parser():
    parser_table = {}
    scanner = None
    stack = []
    reduced = []
    
    #debug apenas
    lastReduce = None
    lastRuleReduce = None
    
    def __init__(self, scanner):
        self.scanner = scanner
        self.scanner.restartAnalisys()
        self.loadTable()
        self.resetStack()
        # Pega sXX ou rXX t.q. XX = id do estado ex: s10 r20  
        self.shiftReduceRegex = re.compile('(s|r)(\d+)')
        
    def errorReport(self, state, expected, cursor):
        if(errors[state]):
            print("Erro: {} mas foi encontrado {} em linha {} coluna {}".format(errors[state], expected, cursor[0]+1, cursor[1]))
        else:
            print("Token inesperado: {} em linha {} coluna {}".format(expected, cursor[0]+1, cursor[1]))
        
    def errorRecovery(self, symbol, state, a, cursor):
#        Descartam-se os símbolos de entrada até encontrar-se um
#        elemento de Follow(A), quando também descartamos A
#        follow
        self.errorReport(state, a.lexema, cursor)
        token = self.scanner.getToken()
        print("debug - erro: ")
        while(token.get() is not 'EOF' and self.getAction(token.get(), state)['err']):
            print("debug - descartando: {}".format(token.get()))
            token = self.scanner.getToken()
        print("debug - retornando: {}".format(token.get()))
        return token

        
    def resetStack(self):
        self.stack = []
        self.stack.append('$')
        self.stack.append(0)
        
    def getTopFromStack(self):
#         print(self.stack[len(self.stack) - 1])
        top = self.stack[len(self.stack) - 1]
        if(isinstance(top, (int, float))):
            return int(top)
        else:
            return top
        
    def loadTable(self):
        self.parser_table = pd.read_csv("tabela_sintatica.csv", delimiter=",", names=["inicio","varinicio","varfim","PT_V","id","inteiro","real","lit","leia","escreva","Literal","Num","RCB","OPM","se","AB_P","FC_P","entao","OPR","fimse","fim","EOF","P'","P","V","LV","D","TIPO","A","ES","ARG","CMD","LD","OPRD","COND","CABEÇALHO","EXP_R","CORPO"]) 
        self.parser_table = self.parser_table.where(pd.notnull(self.parser_table), None)
        
    def getAction(self, symbol, state):
        # inicializa retorno como erro
        functionReturn = {}
        functionReturn['err'] = True
        functionReturn['acc'] = False
        functionReturn['action'] = None
        functionReturn['state'] = None
        
        
#         print("------- debug -------")
#         print("symbol: {} state: {}".format(symbol, state))
        
        tableResult = self.parser_table[symbol][int(state)]
        
        # Se a entrada [symbol][state] da tabela não for vazia
        # Se for None n entra e retorna erro
        if(tableResult):
#             print("------- debug -------")
#             print("tableResult: {}".format(tableResult))
            
            # Pega resultado e quebra em aXX t.q. a=acao XX=estado
            tableMatch = self.shiftReduceRegex.match(str(tableResult))
            
#             print("tableMatch: {}".format(tableMatch))
            # Se der o match no regex é ou shift ou reduce, então adiciona a acao e o estado, e desativa erro 
            functionReturn['err'] = False
            if(tableMatch):
                functionReturn['action'] = tableMatch.group(1)
                functionReturn['state'] = int(tableMatch.group(2))
            else:
                # Se não der match é accept ou um go to
                if isinstance(tableResult, (int, float)):
                    # Se for numérico é go to
                    functionReturn['state'] = int(tableResult)
                else:
                    functionReturn['acc'] = True                
                
        return functionReturn
    
    def parse(self):
        self.resetStack()
        # pede token pro lexico
        a = self.scanner.getToken()
        
        # Inicializa variaveis
        # seja s o estado no topo da pilha
        s = None
        # B = None
        # estado para fazer shift ou estado do goto no reduce
        t = None
        # lado esquerdo da produção no reduce
        A = None
        
        while True:
            # debug imprime topo da pilha
            print(self.stack)
            # Pega topo da pila
            s = self.getTopFromStack()
            
            # Pega acao
            action = self.getAction(a.token, s)
            
            # Checa acoes
            if action['action'] is "s":
                # shift
                t = action['state']
                
                # empilha t
                self.stack.append(t)
#                 print("shift: token {} s".format(a.token, s))
#                 print(a.token)
                
                # pega proximo token
                a = self.scanner.getToken()

            elif action['action'] is "r":
                # reduce
                
                # Remove cardinalidade do lado direito da regra da pilha
                for item in range(grammar[action['state']]['length']):
                    self.stack.pop()
#                     print("debug: desempilha: {}".format(self.stack.pop()))
                    
                # Pega o topo da pilha
                t = self.getTopFromStack()
                
                # Empilha go to
                ruleId = action['state']
                goto = self.getAction(grammar[ruleId]['left'], t)['state']
                self.stack.append(goto)
                
                # Salva ultimo não terminal DEBUG
                self.lastReduce = grammar[ruleId]['left']
                self.lastRuleReduce = rules[ruleId]
                
                # Imprime produção
                print("reduce: {}".format(rules[ruleId]))
                self.reduced.append(rules[ruleId])
                
                
#                 ruleToReduce = grammar[action['state']]

            elif action['acc'] is True:
                # accept
                print("accept")
                break
            else: 
                #reject
#                 print("Chamando rotina de recuperação de erro sintático ... ")
#                 print("debug:  error on {} com token {} e regra {}".format(self.lastReduce, a.get(), self.lastRuleReduce))
                a = self.errorRecovery(self.lastReduce, s, a, self.scanner.getCursorPos())
                if a.get() is 'EOF':
                    break
                
#                 print("error recovery done")

In [20]:
follow

{"P'": ['EOF'],
 'P': ['EOF'],
 'V': ['fim', 'leia', 'escreva', 'id', 'se'],
 'LV': ['fim', 'leia', 'escreva', 'id', 'se'],
 'D': ['varfim', 'id'],
 'TIPO': ['PT_V'],
 'A': ['EOF'],
 'ES': ['fim', 'leia', 'escreva', 'id', 'se', 'fimse'],
 'ARG': ['PT_V'],
 'CMD': ['fim', 'leia', 'escreva', 'id', 'se', 'fimse'],
 'LD': ['PT_V'],
 'OPRD': ['opm', 'PT_V', 'opr', 'FC_P'],
 'COND': ['fim', 'leia', 'escreva', 'id', 'se', 'fimse'],
 'CABEÇALHO': ['leia', 'escreva', 'id', 'fimse', 'se'],
 'EXP_R': ['FC_P'],
 'CORPO': ['fim', 'leia', 'escreva', 'id', 'se', 'fimse']}

Main
========

In [21]:
# Instancia autômato finito para análise léxica
d = DFA(states, alphabet, tf, start_state, accept_states);

# Instancia scanner
scanner = Scanner(d)

# Reinicia analise léxica
scanner.restartAnalisys()

# Instancia parser
parser = Parser(scanner)

parser.parse()

['$', 0]
['$', 0, 2]
['$', 0, 2, 4]
['$', 0, 2, 4, 18]
['$', 0, 2, 4, 18, 39]
reduce: TIPO -> lit
['$', 0, 2, 4, 18, 36]
['$', 0, 2, 4, 18, 36, 51]
reduce: D -> id TIPO ;
['$', 0, 2, 4, 16]
['$', 0, 2, 4, 16, 18]
['$', 0, 2, 4, 16, 18, 37]
reduce: TIPO -> int
['$', 0, 2, 4, 16, 18, 36]
['$', 0, 2, 4, 16, 18, 36, 51]
reduce: D -> id TIPO ;
['$', 0, 2, 4, 16, 16]
['$', 0, 2, 4, 16, 16, 18]
['$', 0, 2, 4, 16, 16, 18, 37]
reduce: TIPO -> int
['$', 0, 2, 4, 16, 16, 18, 36]
['$', 0, 2, 4, 16, 16, 18, 36, 51]
reduce: D -> id TIPO ;
['$', 0, 2, 4, 16, 16, 16]
['$', 0, 2, 4, 16, 16, 16, 18]
['$', 0, 2, 4, 16, 16, 16, 18, 38]
reduce: TIPO -> real
['$', 0, 2, 4, 16, 16, 16, 18, 36]
['$', 0, 2, 4, 16, 16, 16, 18, 36, 51]
reduce: D -> id TIPO ;
['$', 0, 2, 4, 16, 16, 16, 16]
['$', 0, 2, 4, 16, 16, 16, 16, 17]
['$', 0, 2, 4, 16, 16, 16, 16, 17, 35]
reduce: LV -> varfim ;
['$', 0, 2, 4, 16, 16, 16, 16, 34]
reduce: LV -> D LV
['$', 0, 2, 4, 16, 16, 16, 34]
reduce: LV -> D LV
['$', 0, 2, 4, 16, 16, 34]

In [22]:
for rule in parser.reduced:
    print(rule)

# for rule in parser.reduced[::-1]:
#     print(rule)

TIPO -> lit
D -> id TIPO ;
TIPO -> int
D -> id TIPO ;
TIPO -> int
D -> id TIPO ;
TIPO -> real
D -> id TIPO ;
LV -> varfim ;
LV -> D LV
LV -> D LV
LV -> D LV
LV -> D LV
V -> varinicio LV
ARG -> literal
ES -> escreva ARG ;
ES -> leia id ;
ARG -> literal
ES -> escreva ARG ;
ES -> leia id ;
OPRD -> id
OPRD -> num
EXP_R -> OPRD opr OPRD
CABEÇALHO -> se ( EXP_R ) então
OPRD -> id
OPRD -> num
LD -> OPRD opm OPRD
CMD -> id rcb LD ;
OPRD -> id
OPRD -> num
LD -> OPRD opm OPRD
CMD -> id rcb LD ;
OPRD -> id
OPRD -> num
LD -> OPRD opm OPRD
CMD -> id rcb LD ;
OPRD -> id
LD -> OPRD
CMD -> id rcb LD ;
OPRD -> num
LD -> OPRD
CMD -> id rcb LD ;
ARG -> literal
ES -> escreva ARG ;
ARG -> id
ES -> escreva ARG ;
ARG -> literal
ES -> escreva ARG ;
ARG -> id
ES -> escreva ARG ;
ARG -> literal
ES -> escreva ARG ;
ARG -> id
ES -> escreva ARG ;
